In [2]:
###########LIBRARY CELL###########
# pip install watermark
%load_ext watermark
%watermark -v -m -p numpy,scipy,pandas,matplotlib,sklearn -g 
#,statsmodels

CPython 3.6.8
IPython 7.7.0

numpy 1.17.0
scipy 1.3.1
pandas 0.25.0
matplotlib 3.1.1
sklearn 0.21.3

compiler   : GCC 8.0.1 20180414 (experimental) [trunk revision 259383
system     : Linux
release    : 4.9.93-boot2docker
machine    : x86_64
processor  : x86_64
CPU cores  : 2
interpreter: 64bit
Git hash   : 85e16f8d96fcc7fa38e2ae7f1287ada7894aff7f


In [3]:
###########LIBRARY CELL###########
from __future__ import division, print_function
import warnings
warnings.filterwarnings('ignore')
import os
import numpy as np
import pandas as pd
import math
import datetime
import re
import sys

In [40]:
###########VERSION OF COMPARATOR FOR SD<->SD Comparison
###########LIBRARY CELL###########
COMPARISON_TEXT_TYPE = "SimadynD <=> SimadynD"
COMPARATOR_AUTHOR = "Anton Tushev" 
COMPARATOR_VERSION = "v.33.SDSD"
PATH_TO_DATA = '/notebooks/pinda/data'
PATH_TO_STRUCG_1 = '/notebooks/pinda/data/eko/test_1'
PATH_TO_STRUCG_2 = '/notebooks/pinda/data/eko/test_2'
#PATH_TO_STRUCG_1 = '/notebooks/pinda/data/eko/190725'
#PATH_TO_STRUCG_2 = '/notebooks/pinda/data/eko/191031'
PATH_TO_REPORTS = '/notebooks/pinda/reports'
SD_RACK_NAMES = ['sx01']
FILE_1_NAME_TEMP1 = '@sd1_data_s1_progress_bkup.csv'
FILE_1_NAME_TEMP2 = '@sd1_data_ls1_progress_bkup.csv'
FILE_1_NAME_TEMP3 = '@sd1_data_lm_progress_bkup.csv'
FILE_2_NAME_TEMP1 = '@sd2_data_s1_progress_bkup.csv'
FILE_2_NAME_TEMP2 = '@sd2_data_ls1_progress_bkup.csv'
FILE_2_NAME_TEMP3 = '@sd2_data_lm_progress_bkup.csv'
IGNORE_CHART = ['@SIMD']
###Supports ENG,DE
SIMADYND_LANG = "DE" 
SKIP_CHART_NFND = False
SKIP_BLOCK_NEXST = False
SKIP_SIGNAL_NFND = False
SKIP_EQUAL = True
SKIP_TYPE_STR = False

COMPARATOR_README = "===================================================================================\
\r\nAdvanced comparator %s software version: %s \r\nAuthor: %s \
\r\nComparison StrucG folder #1: %s\
\r\nComparison StrucG folder #2: %s\
\r\nList of SimadynD subfolders for parsing is %s \
\r\nList of ignored charts is %s \
\r\n===================================================================================\
"%(COMPARISON_TEXT_TYPE, COMPARATOR_VERSION,COMPARATOR_AUTHOR,PATH_TO_STRUCG_1,PATH_TO_STRUCG_2,SD_RACK_NAMES,IGNORE_CHART)
print(COMPARATOR_README)

Advanced comparator SimadynD <=> SimadynD software version: v.31.SDSD 
Author: Anton Tushev 
Comparison StrucG folder #1: /notebooks/pinda/data/eko/test_1
Comparison StrucG folder #2: /notebooks/pinda/data/eko/test_2
List of SimadynD subfolders for parsing is ['sx01'] 
List of ignored charts is ['@SIMD'] 


In [41]:
###########LIBRARY CELL###########
#Generate Simadyn chart structure
def sd_get_chart_list(PATH_TO_STRUCG,SD_RACK_NAMES=SD_RACK_NAMES):
    #filter chart files
    n = re.compile("^(((?!old).)*).((cfp)|(ofp))$")
    file_list=[] 
    file_list = {rack_name:[item for item in os.listdir(os.path.join(PATH_TO_STRUCG,rack_name)) \
        # chart file names filter conditions
        if os.path.isfile(os.path.join(PATH_TO_STRUCG,rack_name, item)) and (n.search(item)) and len(item)<11 \
        ] for rack_name in SD_RACK_NAMES}
    file_list_filtered = dict(file_list)
    for i in file_list:
        for k in file_list[i]:
            chart_name = k.split('.')[0]
            if "%s.cfp"%chart_name in file_list[i]:
                try:
                    file_list_filtered[i].remove("%s.ofp"%chart_name)
                except ValueError:
                    pass  # do nothing
    out_dict = dict()
    for i in file_list_filtered:
        out_dict[i] = dict()
        for k in file_list_filtered[i]:
            out_dict[i][k.split('.')[0]]=os.path.join(PATH_TO_STRUCG,i,k)
    if out_dict:
        return out_dict
    else:
        return -1
    
#Generate Simadyn cpu mapping structure
def sd_get_cpu_charts_list(PATH_TO_STRUCG,SD_RACK_NAMES=SD_RACK_NAMES):
    #filter chart files
    n = re.compile("^((\d+).*)\.(mpn)$")
    file_list=[] 
    file_list=[] 
    file_list = {rack_name:[item for item in os.listdir(os.path.join(PATH_TO_STRUCG,rack_name)) \
        # chart file names filter conditions
        if os.path.isfile(os.path.join(PATH_TO_STRUCG,rack_name, item)) and (n.search(item)) and len(item)<11 \
        ] for rack_name in SD_RACK_NAMES}
    out_dict = dict()
    for i in file_list:
        out_dict[i] = dict()
        for k in file_list[i]:
            out_dict[i][k.split('.')[0]]=os.path.join(PATH_TO_STRUCG,i,k)
    if out_dict:
        return out_dict
    else:
        return -1

#for debug
##output => #sd_chart_list {"tx03":{'chrt_name':file_addrs}}

In [42]:
###########EXECUTION CELL###########
sd1_chart_list = sd_get_chart_list(PATH_TO_STRUCG_1) 
sd1_cpu_charts_list = sd_get_cpu_charts_list(PATH_TO_STRUCG_1) 
sd2_chart_list = sd_get_chart_list(PATH_TO_STRUCG_2) 
sd2_cpu_charts_list = sd_get_cpu_charts_list(PATH_TO_STRUCG_2) 

In [14]:
###########TEST CELL, IGNORE###########
#n = re.search("^(((?!old).)*).((cfp)|(ofp))$", 'pray1___20131208.cfp')
test_txt = []
test_txt.append(" 129	X01 V2 < 0H0,!E2C113.0011&")
test_txt.append(" 325	I1  B1 < VIHW09.Q15&")
test_txt.append("1397	CCI N2 < DMVI04.Y ")
test_txt.append("1447	I3  B1 < VII250.Q9  ,'ENNROLRD' \"SECTION NOT READY FOR ROLLING\"")
test_txt.append("1229	AD  NK = *ADRCO1,'DO5I2X6A'")
test_txt.append(" 100	IS  V2 < $VST1LS\"Limit switches stand movements 1\"")
test_txt.append('  81	IS  V2 < $VISW17')
test_txt.append(" 163	Q1  B1 > ,'Man__CPS'            \"MAINTENANCE COIL PREP STATION\"")
test_txt.append(' 115	I1  B1 < VIHW17.Q2&')
test_txt.append(' 519	X1  N2 < @TYP=V2,0HFFFF')
test_txt.append(" 150	Y   N2 > @TYP=O2,'RP_ALL'&")
test_txt.append(' 520	X2  N2 < @TYP=V2,LB2.QS&')
test_txt.append(' 527	QS  V2 > $VGAU02                "Diagnose 2nd direction"')
test_txt.append(' 528	Q   B1 > ')
test_txt.append("  31	AD  NK = D02_I1.X8C")
test_txt.append("  32	DM  B1 - 1")
test_txt.append("  22	AR  NS - 'B1G1X1'")
test_txt.append("  21	CTS CR - D0300A")
test_txt.append("1151	T1  NS - '@F@85@ EN QFC Coil P. Shear Tab. Air Float'")
test_txt.append("1199	X1  N2 < 0%")
test_txt.append("1200	X2  N2 < SEND07.QS")
test_txt.append("1201	I   B1 < EN_MES.Q,'EN_MES'&")
test_txt.append(" 566	LZU NF < #10000[ms],'RunTime '  \"Run time monitoring\"")
test_txt.append("1207	X2  N2 < 18.298%                \"18,3 representa 3000 kg\"")
test_txt.append("1283	X2  NF < #0")
test_txt.append("1304	X2  NF < #26,'GRAUS'")
test_txt.append(" 905	X2  NF < #100.0000000E3")
test_txt.append("1074	Y   NF > @TYP=TF,$PTTI_A,'PTTI_AUT'&")
test_txt.append("1073	X   NF < @TYP=TF,10[S]")
test_txt.append("2983	X2  NF < #530.0000000E-3,'m'")
test_txt.append(" 439	FBU NF < #500[ms],'Feedback'    \"Feedback monitoring\"")
test_txt.append(" 494	I   B1 < STG641.Q")
test_txt.append(" 495	Y   N2 > @TYP=V2")
test_txt.append("  17	X1  NF < $XIHW05,SCAL=1[m]")
test_txt.append(" 374	T   TF <  3           [s]")
test_txt.append(" 169	T   TF <            2[s ]")
test_txt.append("2946	OR  V2 < 0B00000000 00000000")
test_txt.append(" 108	X05 NF < #2500.0[V/(m/s)],'ky'  \"Scaling factor for control output\"&")
test_txt.append("2756	CRT TR = !EXM504")
test_txt.append(" 337	Q1  B1 >")
test_txt.append("1407	T   TF < #2.5 [s]")
test_txt.append("  87	Q3  B1 > $E_STOP PN,'E_STOP'&")
test_txt.append(" 267	X   NF < #1E+6")
test_txt.append(' 602	X2  NF < #-0.200E-09[m/(N*s)]   "Umrechnung [N] auf [m]"')
test_txt.append(' 108	TD  TF < #1E-3          [S]      "Verzoegerungszeit"')
test_txt2 = " 439	FBU NF < #500[ms],'Feedback'    \"Feedback monitoring\""


#m = re.compile(r"^\s*\d+\s+(\w+)\s+(\w+)\s+([<>])\s+(@TYP=(..),)?([^@\"',&]+)([,\"'&]|$)", )
m = re.compile(r"""
        ^\s*\d+\s+ #string number
        (\w+)\s+   #signal name 
        (\w+)\s+   #signal type 
        ([<>=-])\s*  #signal delimiter 
        (@TYP=(..),?)? #singal type
        (   #value
        ([a-zA-Z0-9_.]+)|  #can be connection to other block
        ([$%!\[\]/()a-zA-Z0-9\s_.*#\-\+]+)|  #can be value
        ([$a-zA-Z0-9_]+\s*[$a-zA-Z0-9_]*)|  #can be virtual connection
        ([#0-9.]+\s*\[.*\])|  #can be time with spaces
        ([0-9B]+\s+[0-9]+)|  #can be hex in bit representation
        ('.*')| #can be text
        (\*[a-zA-Z0-9_.]+)  #can hardware connection: *ADRCO1
        )? #end of value
        ([\s]*,)? #divider
        (\![a-zA-Z0-9_\.]+)?
        ([\s]*,)?
        ('.*')? #link comment
        (,.*)? #some comment
        (\s*".*")? #signal comment
        (&)? #check next string
        (\s)*
        ($) #end of string
        """, re.X)
k = re.compile(r"^\s*\d+\s+(\w+)\s+(\w+)\s+([<>=-])")

for a in test_txt:
    n = m.search(a)
    print(a)
    #print(n.group(1),n.group(2),n.group(3),n.group(5),n.group(6),n.group(13))
    #print(n)
    print(n.group(6),n.group(15))
    #print(n)
    
#n = m.search(test_txt2)
#print("OUT:",n.group(1),n.group(2),n.group(3),n.group(6),n.group(13))

#for a in test_txt:
#    n = k.search(a)
#    print(n)

#for a in test_txt:
#    n = k.search(a)
#    print(n)

 129	X01 V2 < 0H0,!E2C113.0011&
0H0 !E2C113.0011
 325	I1  B1 < VIHW09.Q15&
VIHW09.Q15 None
1397	CCI N2 < DMVI04.Y 
DMVI04.Y None
1447	I3  B1 < VII250.Q9  ,'ENNROLRD' "SECTION NOT READY FOR ROLLING"
VII250.Q9 None
1229	AD  NK = *ADRCO1,'DO5I2X6A'
*ADRCO1 None
 100	IS  V2 < $VST1LS"Limit switches stand movements 1"
$VST1LS None
  81	IS  V2 < $VISW17
$VISW17 None
 163	Q1  B1 > ,'Man__CPS'            "MAINTENANCE COIL PREP STATION"
None None
 115	I1  B1 < VIHW17.Q2&
VIHW17.Q2 None
 519	X1  N2 < @TYP=V2,0HFFFF
0HFFFF None
 150	Y   N2 > @TYP=O2,'RP_ALL'&
'RP_ALL' None
 520	X2  N2 < @TYP=V2,LB2.QS&
LB2.QS None
 527	QS  V2 > $VGAU02                "Diagnose 2nd direction"
$VGAU02                 None
 528	Q   B1 > 
None None
  31	AD  NK = D02_I1.X8C
D02_I1.X8C None
  32	DM  B1 - 1
1 None
  22	AR  NS - 'B1G1X1'
'B1G1X1' None
  21	CTS CR - D0300A
D0300A None
1151	T1  NS - '@F@85@ EN QFC Coil P. Shear Tab. Air Float'
'@F@85@ EN QFC Coil P. Shear Tab. Air Float' None
1199	X1  N2 < 0%
0% None
1200	

In [13]:
###########TEST CELL, IGNORE###########
test_txt=(" 129	X01 V2 < 0H0,!E2C113.0011&")
m = re.compile(r"""
        ^\s*\d+\s+ #string number
        (\w+)\s+   #signal name 
        (\w+)\s+   #signal type 
        ([<>=-])\s*  #signal delimiter 
        (@TYP=(..),?)? #singal type
        (   #value
        ([a-zA-Z0-9_.]+)|  #can be connection to other block
        ([$%!\[\]/()a-zA-Z0-9\s_.*#\-\+]+)|  #can be value
        ([$a-zA-Z0-9_]+\s*[$a-zA-Z0-9_]*)|  #can be virtual connection
        ([#0-9.]+\s*\[.*\])|  #can be time with spaces
        ([0-9B]+\s+[0-9]+)|  #can be hex in bit representation
        ('.*')| #can be text
        (\*[a-zA-Z0-9_.]+)  #can hardware connection: *ADRCO1
        )? #end of value
        ([\s]*,)? #divider
        (\![a-zA-Z0-9_\.]+)?
        ([\s]*,)?
        ('.*')? #link comment
        (,.*)? #some comment
        (\s*".*")? #signal comment
        (&)? #check next string
        (\s)*
        ($) #end of string
        """, re.X)
a=m.search(test_txt)
print(a.group(15))

!E2C113.0011


In [39]:
###########TEST CELL, IGNORE###########
#test_txt="   &	V4.2.6 FP-COP CHK 22.02.;9 07:54  (MP-TX04.PN-D05_P2 13.09.;6 05:02)"
#test_txt = "   1 PN-D01_P1"
#test_txt = "1832	*"
#test_txt = ""
#test_txt = "  89 Connector list of the function packets"
test_txt = " 402	   $WWSTAT T5 <  FL    PN-D01_P1"
#test_txt = " 830	   $WWSTAT T4 <  PN     PN-D01_P1"
#test_txt = "1816	FP-@SEND1,COM"
#test_txt = "1679	   $VINS02 T4 < PN     PN-D01_P1"
#test_txt = " 830	   $DIADE  T4 < PN     PN-D01_P1"
#test_txt = "ENMESS.Q1231"

#cp = re.compile(r'^\s+\d+\s+([a-zA-Z0-9_-]+)$')
#cp = re.compile(r'^\s+\d+\s+Connector list of the function packets')
#cp = re.compile(r'^\s*\d+\s+\*')
#cp = re.compile(r'^\s*\d+\s+FP-([a-zA-Z0-9_\-@]+)')
cp = re.compile(r'^\s*\d+\s+(\$[a-zA-Z0-9_\-@]+)\s+([\w]+)\s+((<)|(>))\s+([\w]+)\s+([a-zA-Z0-9_-]+)$')
#cp = re.compile(r'^[a-zA-Z0-9_\-@]+\.[a-zA-Z0-9_\-@]+$')
cp_s = cp.search(test_txt)
print(cp_s)
#print(cp_s.group(1),cp_s.group(3),cp_s.group(7))

<_sre.SRE_Match object; span=(0, 37), match=' 402\t   $WWSTAT T5 <  FL    PN-D01_P1'>


In [123]:
###########TEST CELL, IGNORE###########
test_txt = []
test_txt.append("   &		(SEND01.I9)")
test_txt.append('   &		(KA1014.I2,KA1015.I2)')
k = re.compile(r"^\s*&+\s+\((.*)\)\s*$")
for a in test_txt:
    n = k.search(a)
    print(n.group(1))
    #print(n)

SEND01.I9
KA1014.I2,KA1015.I2


In [2]:
###########LIBRARY CELL###########
#########################SimadynD row data generating library#################
NAME,TYPE_BASE,INOUT,TYPE_CONV,VALUE,CPU,CPU_SOURCE,CPU_LINK,CHART,BLOCK,LINKS  = 0,1,2,3,4,5,6,7,8,9,10

#open chart for reading
def sd_open_chart(chart_addr):
    try:
        with open(chart_addr) as f:
            lines = [line.rstrip('\n') for line in f]
    except EnvironmentError: # parent of IOError, OSError *and* WindowsError where available
        #print('ERROR File not exists!')
        return -1
    return lines

#generate chart mapping: line {block name:[block line begin, block line end]}
def sd_get_chart_map(inp_line):
    b = re.compile(r"^\s*\d+ ?([a-zA-Z0-9_]+) +: [a-zA-Z0-9_.@]+\s*,\s*POS=")
    el = re.compile(r'^ *\d+\s+[+]')
    #cp = re.compile(r'\(([a-zA-Z0-9_-]+).([a-zA-Z0-9_-]+)\s+\S+\s+\S+\)') #not used, because not all charts consist
    block_list = []
    mark_bloc = False
    #mark_cpu = False
    out_dict = dict()
    line_counter = 0
    for k in inp_line:
        #Find rack name and cpu name, e.g. MP-TX04 PN-D05_P2
        #if not mark_cpu:
        #    cp_s = cp.search(k)
        #    if cp_s :
        #        rack,cpu = cp_s.group(1),cp_s.group(2)
        #        mark_cpu = True
        b_s = b.search(k)
        if b_s :
            mark_bloc = True
            curr_block_name = b_s.group(1)
            block_list.append(curr_block_name)
            mark_bloc_line_beg = line_counter
        elif el.search(k):
            if mark_bloc:
                mark_bloc = False
                out_dict[curr_block_name] = [mark_bloc_line_beg,line_counter]
        line_counter = line_counter+1        
    #print(len(block_list))
    if(out_dict):
        return out_dict
    else:
        return -1

#check if line is signal or link
def sd_check_line_signal(line):
    sf = re.compile(r"^\s*\d+\s+(\w+)\s+(\w+)\s+([<>=-])")
    sa = re.compile(r"^\s*&+\s+(\(.*\))\s*$")
    if sf.search(line):
        out = 1 #signal
    elif sa.search(line):
        out = 2 #link
    else:
        out = 0 #nothing
    return out

#extract data from signal line
def sd_get_line_data(line): 
    m = re.compile(r"""
        ^\s*\d+\s+ #string number
        (\w+)\s+   #signal name 
        (\w+)\s+   #signal type 
        ([<>=-])\s*  #signal delimiter 
        (@TYP=(..),?)? #singal type
        (   #value
        ([a-zA-Z0-9_.]+)|  #can be connection to other block
        ([$%!\[\]/()a-zA-Z0-9\s_.*#\-\+]+)|  #can be value
        ([$a-zA-Z0-9_]+\s*[$a-zA-Z0-9_]*)|  #can be virtual connection
        ([#0-9.]+\s*\[.*\])|  #can be time with spaces
        ([0-9B]+\s+[0-9]+)|  #can be hex in bit representation
        ('.*')| #can be text
        (\*[a-zA-Z0-9_.]+)  #can hardware connection: *ADRCO1
        )? #end of value
        ([\s]*,)? #divider
        (\![a-zA-Z0-9_\.]+)?
        ([\s]*,)?
        ('.*')? #link comment
        (,.*)? #some comment
        (\s*".*")? #signal comment
        (&)? #check next string
        (\s)*
        ($) #end of string
        """, re.X)
    #print(line)
    n = m.search(line)
    try:
        #Check if value is XXXX but behind is telegram connection like !XXX.YY
        if n.group(15):
            check_val = n.group(15)
        else:
            check_val = n.group(6)
        out = list([n.group(1),n.group(2),n.group(3),n.group(5),check_val])
    except:
        #print line for debugging
        print(line) 
        raise
        
    #     #name      type       dir        type_conv  value 
    #list([n.group(1),n.group(2),n.group(3),n.group(5),n.group(6)])
    return out

#exstract link from like
def sd_get_line_link(line): 
    m = re.compile(r"^\s*&+\s+\((.*)\)\s*$")
    n = m.search(line)     
    return n.group(1)  

#get block data: signals, values, links, data types...
def sd_get_block_data(lines,chart,block,data_link_map_df):
    mark_signal_row_name = None
    mark_signal_row_data = list([[],[]]) #[[signal data],[links]]
    mark_block_row_data = []
    count = 0;
    #get CPU name
    cpu = sd_get_chart_cpu(data_link_map_df,chart)
    #If no cpu found, it means chart is not used in SD software:
    if cpu == -1:
        return -1
    for l in lines:
        line_check_result = sd_check_line_signal(l)
        if line_check_result:
            if line_check_result==1:  
                #if found a signal line
                count = count+1
                #add previous signal to preparation table
                if mark_signal_row_name:
                    #print("Old name:",mark_signal_row_name)
                    mark_block_row_data.append(mark_signal_row_data)
                    mark_signal_row_data = list([[],[]]) #reset buffer table
                #NAME,TYPE_BASE,INOUT,TYPE_CONV,VALUE
                mark_signal_row_data[0] = sd_get_line_data(l)
                mark_signal_row_data[0].append(cpu)
                #If value is $LINK here is a logic:
                #Copy value $LINK to link column
                if re.search(r'^\$',str(mark_signal_row_data[0][VALUE])) and mark_signal_row_data[0][INOUT]=="<":
                    link = mark_signal_row_data[0][VALUE].split()[0] #CONVERT '$LINK PN' => '$LINK'
                    mark_signal_row_data[1].append(link)
                    #get CPU_SOURCE
                    cpu_source = sd_get_block_link_source_cpu(data_link_map_df,link,chart)
                    mark_signal_row_data[0].append(cpu_source)
                    #get CPU_LINK   (means CPU_SOURCE_LINK)
                    mark_signal_row_data[0].append(cpu_source+"_"+link)
                else:
                    #get CPU_SOURCE
                    mark_signal_row_data[0].append(cpu)
                    #get CPU_LINK   (means CPU_SOURCE_LINK)
                    mark_signal_row_data[0].append(None)
                    
                mark_signal_row_name = mark_signal_row_data[0][NAME]
                #get CHART,BLOCK
                mark_signal_row_data[0].append(chart)
                mark_signal_row_data[0].append(block)
            elif line_check_result==2:
                #if found a link line: (SOMETHING,SOMETHING2)
                mark_signal_row_data[1].extend(sd_get_line_link(l).split(','))
            #print(mark_signal_row_data)            
        else:
            pass #skip string
    #Add last signal to preparation storage
    mark_block_row_data.append(mark_signal_row_data)
    return mark_block_row_data

#convert list to pandas data frame
def sd_conv_block_data_to_df(block_data):
    #list element of row signal data:[[NAME,TYPE_BASE,INOUT,TYPE_CONV,VALUE],[links]]
    block_df = pd.DataFrame([[j for j in k[0]]+[k[1]] for k in block_data])
    return block_df
#convert list to better representation
def sd_conv_block_data_to_list(block_data):
    #list element of row signal data:[[NAME,TYPE_BASE,INOUT,TYPE_CONV,VALUE,CPU,CPU_SOURCE,CPU_LINK,CHART,BLOCK],[links]]
    block_data_list = [[j for j in k[0]]+[k[1]] for k in block_data]
    return block_data_list


#create df row with link source structure: chart_block_signal,cpu_link
def sd_get_source_link(block_data_list):
    #block_data_list: (NAME,TYPE_BASE,INOUT,TYPE_CONV,VALUE,CPU,CPU_SOURCE,CPU_LINK,CHART,BLOCK ),LINKS
    m = re.compile(r"^\$\w+")
    out_df = []
    for j in block_data_list:
        if j[INOUT]==">":
            if(j[INOUT]):
                
                val = str(j[VALUE])
                n = m.search(val) 
                if n:
                    #chart\block.signal,cpu_link
                    chart_block_signal = j[CHART].upper()+'\\'+j[BLOCK].upper()+'.'+j[NAME].upper()
                    cpu_link = j[CPU]+'_'+val.split()[0] #convert '$LINK PN' => '$LINK'
                    #print(chart_block_signal,"=>",cpu_link)
                    out_df.append([chart_block_signal,cpu_link])
            #look for source link in link array
            for k in j[LINKS]:
                if(k):
                    n = m.search(str(k)) 
                    if n:
                        #chart\block.signal,cpu_link
                        chart_block_signal = j[CHART].upper()+'\\'+j[BLOCK].upper()+'.'+j[NAME].upper()
                        cpu_link = j[CPU]+'_'+str(k).split()[0] #convert '$LINK PN' => '$LINK'
                        #print(chart_block_signal,"=>",cpu_link)
                        out_df.append([chart_block_signal,cpu_link])
    return out_df

#generate data of all charts and generate data of link sources
def sd_build_data_s1(sd_chart_list,data_link_map_df):
    
    data_list = []
    data_link_source_list = []
    for i in sd_chart_list:
        for (k,v) in sd_chart_list[i].items(): 
            chart = k.upper()
            chart_lines = sd_open_chart(v)
            chart_mapping = sd_get_chart_map(chart_lines)
            if chart_mapping==-1:
                print("Parsing '%s' chart with 0 functions"%(chart))
                continue
            else:
                print("Parsing '%s' chart with %s functions"%(chart,len(chart_mapping)))
            
            for block_map in chart_mapping:
                #generating array 'signal','type_base','inout','type_conv','value','link','chart','block','cpu'
                block_lines = chart_lines[chart_mapping[block_map][0]:chart_mapping[block_map][1]]
                #get block data: NAME,TYPE_BASE,INOUT,TYPE_CONV,VALUE,CPU,CPU_SOURCE,CPU_LINK,CHART,BLOCK
                block_data = sd_get_block_data(block_lines,chart,block_map,data_link_map_df)
                if block_data == -1:
                    print("Parsing cancelled: '%s' chart is not used in SimadynD program"%(chart))
                    break
                #print("before ",block_data)
                #print("-- ")
                block_data_list = sd_conv_block_data_to_list(block_data)
                #print("after ",block_data_list)
                #print("-- ")
                #Add block data to accumulation list
                data_list.extend(block_data_list)
                #create link source array
                block_link_source_list = sd_get_source_link(block_data_list)
                if len(block_link_source_list)!=0:
                    #print(block_link_source_list)
                    data_link_source_list.extend(block_link_source_list) 
    #Convert to DataFrame for easy work
    data_link_source_df = pd.DataFrame(data_link_source_list, columns=['chart_block_signal','cpu_link'])
    data_df = pd.DataFrame(data_list, columns=['signal','type_base','inout','type_conv','value','cpu',\
                                                         'cpu_source','cpu_link','chart','block','link'])
    return data_df,data_link_source_df

#parsing mapping file
#generate cpu,link db by cpu mapping file
def sd_build_cpu_map(sd_chart_list):
    data_link_map_list = []
    bg_lang_options = {"EN":"Connector list of the function packets",
                       "DE":"Konnektorliste der Funktionspakete"}
    cp = re.compile(r'^\s+\d+\s+([a-zA-Z0-9_-]+)$') #find CPU name in file
    bg = re.compile(r'^\s+\d+\s+%s'%(bg_lang_options[SIMADYND_LANG])) # find beginning of link information
    en = re.compile(r'^\s*\d+\s+\*')
    ch = re.compile(r'^\s*\d+\s+FP-([a-zA-Z0-9_\-@]+)')
    lk = re.compile(r'^\s*\d+\s+(\$[a-zA-Z0-9_\-@]+)\s+([\w]+)\s+((<)|(>))\s+([\w]+)\s+([a-zA-Z0-9_-]+)$')
    
    for i in sd_chart_list:
        for (k,v) in sd_chart_list[i].items():
            chart_lines = sd_open_chart(v)
            print("Parsing cpu file '%s'  "%(k))
            cpu_mark = False
            beg_mark = False
            end_mark = False
            for l in chart_lines:
                ##Find CPU name
                if not cpu_mark:
                    cp_s = cp.search(l)
                    if cp_s :
                        cpu = cp_s.group(1)
                        cpu_mark = True
                        print(cpu)   
                else:
                    if not beg_mark:
                        if bg.search(l):
                            beg_mark = True
                    else:
                        #check end
                        if en.search(l):
                            end_mark = True
                            break
                        else:
                            #main block:
                            ch_s = ch.search(l)
                            if ch.search(l): #if chart beginning 1739	FP-MONIW1
                                chart = ch_s.group(1)
                            else:
                                lk_s = lk.search(l)
                                if lk_s: #if chart link definition 1744	   $DMALB1 T5 < PN     PN-D01_P1
                                    #'cpu','chart','out','link','cpu_source'
                                    data_link_map_list.append([cpu, chart.upper(),lk_s.group(3),lk_s.group(1),lk_s.group(7)])
    data_link_map_df = pd.DataFrame(data_link_map_list, columns=['cpu','chart','out','link','cpu_source'])
    return data_link_map_df

def sd_get_chart_cpu(data_link_map_df,chart):
    try:
        result = data_link_map_df[data_link_map_df['chart']==chart].iloc[0]['cpu']
    except IndexError:
        result = -1
    return result
def sd_get_block_link_source_cpu(data_link_map_df,link,chart):
    print("debug:",link,chart)
    return data_link_map_df[(data_link_map_df['chart']==chart)&\
                            (data_link_map_df['link']==link)&\
                            (data_link_map_df['out']=="<")].iloc[0]['cpu_source']

def sd_get_link_partner(data_link_source_df,cpu_link):
    try:
        return data_link_source_df[(data_link_source_df['cpu_link']==cpu_link)].iloc[0]['chart_block_signal']
    except IndexError:
        return None

In [44]:
###########EXECUTION CELL###########
#creating row data DataFrame "sd1_data_s1" from chart list
sd1_data_lm = sd_build_cpu_map(sd1_cpu_charts_list)
sd1_data_s1,sd1_data_ls1 = sd_build_data_s1(sd1_chart_list,sd1_data_lm)
sd1_data_s1["block"] = sd1_data_s1["block"].astype(str)

Parsing cpu file '1sx01'  
PN-D01_P1
Parsing cpu file '2sx01'  
PN-D04_P2
Parsing cpu file '3sx01'  
PN-D05_P3
Parsing cpu file '4sx01'  
PN-D06_P4
Parsing cpu file '5sx01'  
PN-D08_P5
Parsing '@RCV_1' chart with 1 functions
Parsing '@SND_1' chart with 14 functions
Parsing '@SND_2' chart with 2 functions
Parsing '@SND_3' chart with 2 functions
Parsing '@SND_4' chart with 2 functions
Parsing '@SND_5' chart with 2 functions
Parsing 'PWR__3' chart with 267 functions
debug: $SETZ12 PWR__3
debug: $SETZ11 PWR__3
debug: $CUID34 PWR__3
debug: $CUID12 PWR__3
debug: $CUID78 PWR__3
debug: $CUID56 PWR__3
debug: $ID1112 PWR__3
debug: $CID910 PWR__3
debug: $WRB5ON PWR__3
debug: $RAC5ON PWR__3
debug: $WVROL5 PWR__3
debug: $WVSYEX PWR__3
debug: $N_ST_H PWR__3
debug: $XTHAVG PWR__3
debug: $N_ST_W PWR__3
debug: $XWIAVG PWR__3
debug: $CLDEVH PWR__3
debug: $CL_DEV PWR__3
debug: $TEMPFS PWR__3
debug: $TEMPDC PWR__3
debug: $XPROFL PWR__3
debug: $F5_FOR PWR__3
debug: $FRBSF5 PWR__3
debug: $FRASF5 PWR__3
debu

In [45]:
sd1_data_lm[sd1_data_lm["link"]=="$WWSTAT"]

,cpu,chart,out,link,cpu_source
50,PN-D01_P1,MMID_1,<,$WWSTAT,PN-D01_P1
318,PN-D01_P1,FXS__1,>,$WWSTAT,PN-D01_P1
350,PN-D01_P1,FXS__1,>,$WWSTAT,PN-D01_P1
2177,PN-D05_P3,PWR__3,<,$WWSTAT,PN-D01_P1


In [29]:
###########EXECUTION CELL###########
#creating row data DataFrame "sd2_data_s1" from chart list
sd2_data_lm = sd_build_cpu_map(sd2_cpu_charts_list)
sd2_data_s1,sd2_data_ls1 = sd_build_data_s1(sd2_chart_list,sd2_data_lm)
sd2_data_s1["block"] = sd2_data_s1["block"].astype(str)

Parsing cpu file '1sx21'  
PN-D01_P1
Parsing cpu file '2sx21'  
PN-D05_P2
Parsing cpu file '3sx21'  
PN-D08_P3
Parsing cpu file '4sx21'  
PN-D11_P4
Parsing cpu file '5sx21'  
PN-D13_P5
Parsing cpu file '6sx21'  
PN-D15_P6
Parsing 'ZHGC_6' chart with 199 functions


In [73]:
###########EXECUTION CELL###########
#save intermidiate result
sd1_data_s1.to_csv(os.path.join(PATH_TO_DATA,FILE_1_NAME_TEMP1))
sd1_data_ls1.to_csv(os.path.join(PATH_TO_DATA,FILE_1_NAME_TEMP2))
sd1_data_lm.to_csv(os.path.join(PATH_TO_DATA,FILE_1_NAME_TEMP3))
sd2_data_s1.to_csv(os.path.join(PATH_TO_DATA,FILE_2_NAME_TEMP1))
sd2_data_ls1.to_csv(os.path.join(PATH_TO_DATA,FILE_2_NAME_TEMP2))
sd2_data_lm.to_csv(os.path.join(PATH_TO_DATA,FILE_2_NAME_TEMP3))


In [7]:
###########EXECUTION CELL###########
###########SKIP###########
#restore intermediate result
sd1_data_s1 = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_1_NAME_TEMP1),index_col=0)
sd1_data_ls1 = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_1_NAME_TEMP2),index_col=0)
sd1_data_lm = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_1_NAME_TEMP3),index_col=0)
sd2_data_s1 = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_2_NAME_TEMP1),index_col=0)
sd2_data_ls1 = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_2_NAME_TEMP2),index_col=0)
sd2_data_lm = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_2_NAME_TEMP3),index_col=0)

In [30]:
###########LIBRARY CELL###########
#find partner link and put in 'value_linked' column.
#This column should be used for comparison
def sd_rect_data_s1(data_df,data_link_source_df):
    def lookup(x):
        partner = sd_get_link_partner(data_link_source_df,x)
        return partner
    data_df['value_linked'] = data_df['cpu_link'].apply(lookup)
    return data_df

#copy values in 'value_linked' - to get proper view: value or link to other block. 
#This column should be used for comparison
#Also convert in-chart link to absolute representation e.g. ENMESS.Q => MONIW1\ENMESS.Q
def sd_rect_data_s2(data_df):
    def lookup(x):
        if x['value_linked'] is None:
            if re.search(r'^[a-zA-Z0-9_\-@]+\.[a-zA-Z0-9_\-@]+$',str(x['value'])) and not re.search(r'^[0-9]+\.[0-9]+$',str(x['value'])):
                if not x['type_base'] in ['CR','NK','TR','RR']:
                    return x['chart']+"\\"+x['value']
                else:
                    return x['value']
            else:
                #convert output 'link' array to string and return to 'value_linked'
                #TODO: doesn't work with $links -> should be extended for this case.
                if x['value'] is None and x['link']:
                    return ','.join(x['link'])
                else:
                    return x['value']  
        else:
            return x['value_linked']
    data_df['value_linked'] = data_df.apply(lookup,axis = 1)
    return data_df



In [31]:
###########EXECUTION CELL###########
#find links for sd1
sd1_data_s2 = sd_rect_data_s1(sd1_data_s1.copy(),sd1_data_ls1)
sd1_data_s3 = sd_rect_data_s2(sd1_data_s2.copy())

In [15]:
sd1_data_s3[(sd1_data_s3['block']=="SOVLS2")]

,signal,type_base,inout,type_conv,value,cpu,cpu_source,cpu_link,chart,block,link,value_linked
413,X,NF,<,None,KOVLSN.Y,PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[],ZHGC_6\KOVLSN.Y
414,LU,NF,<,None,0.000[m],PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[],0.000[m]
415,LL,NF,<,None,-0.000500[m],PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[],-0.000500[m]
416,SV,NF,<,None,0.000[m],PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[],0.000[m]
417,TI,TF,<,None,1[s],PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[],1[s]
418,S,B1,<,None,V4SET2.Q11,PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[],ZHGC_6\V4SET2.Q11
419,Y,NF,>,None,None,PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[WSS.X4],WSS.X4
420,QU,B1,>,None,None,PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[],None
421,QL,B1,>,None,None,PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[],None


In [32]:
###########EXECUTION CELL###########
#find links for sd2
sd2_data_s2 = sd_rect_data_s1(sd2_data_s1.copy(),sd2_data_ls1)
sd2_data_s3 = sd_rect_data_s2(sd2_data_s2.copy())

In [17]:
sd2_data_s3[(sd2_data_s3['block']=="SOVLS2")]

,signal,type_base,inout,type_conv,value,cpu,cpu_source,cpu_link,chart,block,link,value_linked
413,X,NF,<,None,KOVLSN.Y,PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[],ZHGC_6\KOVLSN.Y
414,LU,NF,<,None,0.000[m],PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[],0.000[m]
415,LL,NF,<,None,-0.000500[m],PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[],-0.000500[m]
416,SV,NF,<,None,0.000[m],PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[],0.000[m]
417,TI,TF,<,None,1[s],PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[],1[s]
418,S,B1,<,None,V4SET2.Q11,PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[],ZHGC_6\V4SET2.Q11
419,Y,NF,>,None,None,PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[WSS.X40000],WSS.X40000
420,QU,B1,>,None,None,PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[],None
421,QL,B1,>,None,None,PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[],None


In [251]:
#data_s3[~data_s3['cpu_link'].isnull()]

In [33]:
###########LIBRARY CELL###########
#report generator of comparison of block in all SD1 and SD2 charts
def report_find_miss_blocks (sd1_data_sd,sd2_data_sd, to_file = True, timestamp = datetime.datetime.now()):
    README_HEADER = COMPARATOR_README +"\
\r\nReport: find missing blocks in sources\
\r\nDate ant time of report generation: %s \
\r\n===================================================================================\
\r\n"%(str(timestamp))
    
    def generate_report(f):
        print(README_HEADER,file = f) 
        sd1_chartblock_sel = sd1_data_sd["chart"]+"/"+sd1_data_sd["block"]
        sd1_uniq_blocks = sd1_chartblock_sel.unique()
        #tdc_uniq_blocks = data_tdc['chartblock'].unique()
        sd2_chartblock_sel = sd2_data_sd["chart"]+"/"+sd2_data_sd["block"]
        sd2_uniq_blocks = sd2_chartblock_sel.unique()
        #look for A in B
        for i in sd1_uniq_blocks:
            if not i in sd2_uniq_blocks:
                print("Missing block in SD#2: ",i,"\r\n",file = f) 
        #look for B in A
        for i in sd2_uniq_blocks:
            if not i in sd1_uniq_blocks:
                print("Missing block in SD#1: ",i,"\r\n",file = f) 
  
    if to_file:
        file_suffix = "@_compare_blocks_sd_%s"%(str(timestamp.date()))
        file_prefix = "report_@"
        try:
            # Create target Directory
            os.mkdir(os.path.join(PATH_TO_REPORTS))
        except FileExistsError:
            pass
        #sys.stdout = open(os.path.join(PATH_TO_REPORTS,(file_prefix+FILE_NAME_DATA1.split(".")[0]+file_suffix+".txt")), 'w')
        with open(os.path.join(PATH_TO_REPORTS,(file_prefix+SD_RACK_NAMES[0]+file_suffix+".txt")), 'w') as f: 
            generate_report(f)
    else:
        generate_report(sys.stdout)

In [34]:
#############EXECUTION GENERATE REPORT report_find_miss_blocks############
#report_find_miss_blocks(data_s2,data_a1,to_file = True)
report_find_miss_blocks(sd1_data_s2,sd2_data_s2,to_file = True)

In [35]:
###########LIBRARY CELL###########
#####report of comparison of all SD blocks inputs, including links

        #['BOOL' 'INT' 'WORD' 'SDTIME' 'GLOBAL' 'STRING' 'REAL' 'DINT' 'BYTE' 'DWORD']
        #SD types:
        #'B1' - BOOL (0/1/link)
        #'TF' - Time (ffff.fff[s/min/S/ms])
        #'N2' - Integer (ddddd)
        #'V2' - Integer HEX
        #'MR' - Message link (e.g MESY_1)(Inout = "-")
        #'NS' - String (Inout = "-")
        #'O2' - Integer (dddddd)
        #'CR' - Hardware connectors, string
        #'V4' - Dword HEX
        #'NF' - Floating point
        #'V1' - Byte HEX   
        #'I2' - Integer
        #'NK' - Hardware connectors, string (Inout = "=")
        #'TR' - Telegram transmit connector e.g. !EXM504 (Inout = "=")
        #'N4' - Integer 32 bits (%/HEX/ddd)
        #'RR' - Telegram recieve connector e.g. !T4EX05 (Inout = "=") 
        
#report parameters    
filt_type_base = ['B1','TF','N2','N4','V2','O2','V4','NF','V1','I2','TR','RR']  #,'NS'
filt_inout = ['<','>','=','-']#,'-'

def report_comp_inputs (s1_inp_data,s2_inp_data, to_file = True, timestamp = datetime.datetime.now()):
    README_HEADER = COMPARATOR_README +"\
\r\nReport: compare all SD#1 inputs (including links) with SD#2\
\r\nDate ant time of report generation: %s \
\r\n===================================================================================\
\r\n"%(str(timestamp))
    sd1_data_sd = s1_inp_data.copy()
    #data_tdc = inp_data_tdc.copy()
    sd2_data_sd = s2_inp_data.copy()
    def merge_tdc_link_value(l_value,l_link):
        if pd.isnan(l_link):
            return l_link
        else:
            return l_value
    
    def clean_sd_tdc_str_values(s1_value,s2_value,sd_type,sd_type_conv,chart_block_signal,source_dest_text,f):
        #modify HEX function
        def twos_complement(hexstr,bits,base=16):
            value = int(hexstr,base)
            if value & (1 << (bits-1)):
                value -= 1 << bits
            return value
 
        #for debugging
        #print("Working on:",s_value,a_value,sd_type,sd_type_conv)
        
        #check if values are empty
        if  pd.isnull(s1_value):
            if pd.isnull(s2_value):
                return True
            else:
                result = False;
        else:
            if pd.isnull(s2_value):
                result = False;
            else:
                result = s1_value == s2_value;
            
        if not result:
            print("\r\nDifference found in ",source_dest_text,chart_block_signal,": ",s1_value,'==>>> ',s2_value,file = f)
        return result

    def generate_report(f):
        print(README_HEADER,file = f) 
        #this comparator works with following data types and connectors:
        #filter sd data frame
        sd1_data_sd_filtered = sd1_data_sd[(sd1_data_sd['type_base'].isin(filt_type_base))&(sd1_data_sd['inout'].isin(filt_inout))]
        sd2_data_sd_filtered = sd2_data_sd[(sd2_data_sd['type_base'].isin(filt_type_base))&(sd2_data_sd['inout'].isin(filt_inout))]
        
        sd1_data_sd_filtered['chart_block_signal'] = sd1_data_sd_filtered["chart"]+"/"+sd1_data_sd_filtered["block"]+"."+sd1_data_sd_filtered["signal"]
        sd2_data_sd_filtered['chart_block_signal'] = sd2_data_sd_filtered["chart"]+"/"+sd2_data_sd_filtered["block"]+"."+sd2_data_sd_filtered["signal"]
        
        #connect links $XXXXX->$XXXXX
        #d = dict(zip(data_tdc['chart_block_signal'].values, data_tdc['value_linked'].values)) #for tdc
        d1 = dict(zip(sd2_data_sd_filtered['chart_block_signal'].values, sd2_data_sd_filtered['value_linked'].values))
        d2 = dict(zip(sd1_data_sd_filtered['chart_block_signal'].values, sd1_data_sd_filtered['value_linked'].values))
        
        sd1_data_sd_filtered['val_to_compare'] = sd1_data_sd_filtered['chart_block_signal'].map(d1)
        sd1_data_sd_filtered['compare_val_sd'] = sd1_data_sd_filtered.apply(lambda row: clean_sd_tdc_str_values(\
           row['value_linked'], row['val_to_compare'],row['type_base'], row['type_conv'],row['chart_block_signal'],"SD#1->SD#2 value:",f), axis=1)
        
        sd2_data_sd_filtered['val_to_compare'] = sd2_data_sd_filtered['chart_block_signal'].map(d2)
        sd2_data_sd_filtered['compare_val_sd'] = sd2_data_sd_filtered.apply(lambda row: clean_sd_tdc_str_values(\
           row['value_linked'], row['val_to_compare'],row['type_base'], row['type_conv'],row['chart_block_signal'],"SD#2->SD#1 value:",f), axis=1)
        
    if to_file:
        file_suffix = "@_compare_inputs_sd_%s"%(str(timestamp.date()))
        file_prefix = "report_@"
        try:
            # Create target Directory
            os.mkdir(os.path.join(PATH_TO_REPORTS))
        except FileExistsError:
            pass
        #sys.stdout = open(os.path.join(PATH_TO_REPORTS,(file_prefix+FILE_NAME_DATA1.split(".")[0]+file_suffix+".txt")), 'w')
        with open(os.path.join(PATH_TO_REPORTS,(file_prefix+SD_RACK_NAMES[0]+file_suffix+".txt")), 'w') as f: 
            generate_report(f)
    else:
        generate_report(sys.stdout)
        


In [36]:
#############EXECUTION GENERATE REPORT report_comp_inputs############
#report_comp_inputs(data_s3,data_a1,to_file = True)
report_comp_inputs(sd1_data_s3,sd2_data_s3,to_file = True)


In [23]:
#TEST SELECT FROM SIMADYND TABLE
#NOVLDO
sd2_data_s2[(sd2_data_s3['block']=="NOVLDO")]

,signal,type_base,inout,type_conv,value,cpu,cpu_source,cpu_link,chart,block,link,value_linked
4178,I1,B1,<,None,V_OVLP.Q11,PN-D11_P4,PN-D11_P4,None,XHSET4,NOVLDO,[],None
4179,I2,B1,<,None,V_OVLP.Q13,PN-D11_P4,PN-D11_P4,None,XHSET4,NOVLDO,[],None
4180,Q,B1,>,None,None,PN-D11_P4,PN-D11_P4,None,XHSET4,NOVLDO,[NOVLIM.I],None


In [78]:

sd2_data_s3[(sd2_data_s3['cpu_link']!=None)]

,signal,type_base,inout,type_conv,value,cpu,cpu_source,cpu_link,chart,block,link,value_linked
0,IS,V2,<,None,$V_TAR,PN-D11_P4,PN-D11_P4,PN-D11_P4_$V_TAR,XFMS_4,V_TAR,[$V_TAR],XTAR_4\V_TAR.QS
1,Q1,B1,>,None,None,PN-D11_P4,PN-D11_P4,None,XFMS_4,V_TAR,[],None
2,Q2,B1,>,None,None,PN-D11_P4,PN-D11_P4,None,XFMS_4,V_TAR,[],None
3,Q3,B1,>,None,None,PN-D11_P4,PN-D11_P4,None,XFMS_4,V_TAR,[],None
4,Q4,B1,>,None,None,PN-D11_P4,PN-D11_P4,None,XFMS_4,V_TAR,[],None
5,Q5,B1,>,None,None,PN-D11_P4,PN-D11_P4,None,XFMS_4,V_TAR,[],None
6,Q6,B1,>,None,None,PN-D11_P4,PN-D11_P4,None,XFMS_4,V_TAR,[],None
7,Q7,B1,>,None,None,PN-D11_P4,PN-D11_P4,None,XFMS_4,V_TAR,[],None
8,Q8,B1,>,None,None,PN-D11_P4,PN-D11_P4,None,XFMS_4,V_TAR,"[EMSK05.I2, EN00.I2, EN04.I2, EN05.I2]",None
9,Q9,B1,>,None,None,PN-D11_P4,PN-D11_P4,None,XFMS_4,V_TAR,[],None


In [36]:
sd1_data_sd_filtered = sd1_data_s3[(sd1_data_s3['type_base'].isin(filt_type_base))&(sd1_data_s3['inout'].isin(filt_inout))]       
sd1_data_sd_filtered['chart_block_signal'] = sd1_data_sd_filtered["chart"]+"/"+sd1_data_sd_filtered["block"]+"."+sd1_data_sd_filtered["signal"]

sd2_data_sd_filtered = sd2_data_s3[(sd2_data_s3['type_base'].isin(filt_type_base))&(sd2_data_s3['inout'].isin(filt_inout))]       
sd2_data_sd_filtered['chart_block_signal'] = sd2_data_sd_filtered["chart"]+"/"+sd2_data_sd_filtered["block"]+"."+sd2_data_sd_filtered["signal"]

d1 = dict(zip(sd2_data_sd_filtered['chart_block_signal'].values, sd2_data_sd_filtered['value_linked'].values))
d2 = dict(zip(sd1_data_sd_filtered['chart_block_signal'].values, sd1_data_sd_filtered['value_linked'].values))

sd2_data_sd_filtered['val_to_compare'] = sd2_data_sd_filtered['chart_block_signal'].map(d2)


#sd2_data_sd_filtered[(sd2_data_sd_filtered['value']=="$XPHPOS")&(sd2_data_sd_filtered['cpu_link'])] #&)
sd2_data_sd_filtered[(sd2_data_s3['block']=="SOVLS2")]#(sd2_data_s3['inout']==">")&

,signal,type_base,inout,type_conv,value,cpu,cpu_source,cpu_link,chart,block,link,value_linked,chart_block_signal,val_to_compare
11256,X,NF,<,None,KOVLSN.Y,PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[],ZHGC_6\KOVLSN.Y,ZHGC_6/SOVLS2.X,ZHGC_6\KOVLSN.Y
11257,LU,NF,<,None,0.000[m],PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[],0.000[m],ZHGC_6/SOVLS2.LU,0.000[m]
11258,LL,NF,<,None,-0.000500[m],PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[],-0.000500[m],ZHGC_6/SOVLS2.LL,-0.000500[m]
11259,SV,NF,<,None,0.000[m],PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[],0.000[m],ZHGC_6/SOVLS2.SV,0.000[m]
11260,TI,TF,<,None,1[s],PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[],1[s],ZHGC_6/SOVLS2.TI,1[s]
11261,S,B1,<,None,V4SET2.Q11,PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[],ZHGC_6\V4SET2.Q11,ZHGC_6/SOVLS2.S,ZHGC_6\V4SET2.Q11
11262,Y,NF,>,None,None,PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[WSS.X4],None,ZHGC_6/SOVLS2.Y,None
11263,QU,B1,>,None,None,PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[],None,ZHGC_6/SOVLS2.QU,None
11264,QL,B1,>,None,None,PN-D15_P6,PN-D15_P6,None,ZHGC_6,SOVLS2,[],None,ZHGC_6/SOVLS2.QL,None


In [80]:
sd1_data_lm[sd1_data_lm['link']=="$XPHPOS"]

,cpu,chart,out,link,cpu_source
321,PN-D01_P1,OPD__1,<,$XPHPOS,PN-D15_P6
1327,PN-D11_P4,XFMS_4,<,$XPHPOS,PN-D15_P6
2279,PN-D15_P6,ZHGC_6,>,$XPHPOS,PN-D15_P6
